In [1]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
from os.path import dirname, join

In [2]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [3]:
path = join(dirname(os.getcwd()), "Vision_Final_Project","fotos")
imgs_path = [join(path, f"{img_path}") for img_path in os.listdir(path)]
imgs = load_images(imgs_path)


/var/folders/30/_9yyp9911q3b6fn11_45rt6m0000gn/T/ipykernel_3430/283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [4]:
corners = [cv2.findChessboardCorners(img,(7,7),None) for img in imgs]


In [5]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (7,7), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]


In [6]:
imgs_copy = copy.deepcopy(imgs)


In [7]:

for index, (img, corner) in enumerate(zip(imgs_copy, corners)):
    if corner[0]:
        cv2.drawChessboardCorners(img, (7,7), corner[1], corner[0])
        cv2.imshow(f'Imagen {index} con esquinas', img)
        cv2.waitKey(500)
        cv2.destroyAllWindows()


2024-12-03 18:33:49.431 Python[3430:16316] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [8]:
def show_image(img):
    cv2.imshow('Esquinas del tablero de ajedrez', img)
    cv2.waitKey(500)
    cv2.destroyAllWindows()
    
def write_image(path_name,img):
    cv2.imwrite(path_name,img)

In [9]:
def get_chessboard_points(chessboard_shape, dx, dy):
    # rows, cols = chessboard_shape
    # points = np.zeros((rows*cols,1,3),dtype=np.float32)
    points = []
    for y in range(chessboard_shape[1]):
        for x in range(chessboard_shape[0]):
            # points[i*cols+j] = [j*dx,i*dy,0]
            points.append([x*dx,y*dy,0])
    return np.array(points,np.float32)

In [10]:
chessboard_points = [get_chessboard_points((7,7), 30, 30) for corner in corners_copy if corner[0]]

In [11]:
valid_corners = [cor[1] for cor in corners_copy if cor[0]]
valid_corners = np.asarray(valid_corners, dtype=np.float32)
valid_corners

array([[[[ 864.2804  ,  137.6336  ]],

        [[ 875.8287  ,  198.9104  ]],

        [[ 888.81024 ,  262.15103 ]],

        ...,

        [[ 301.62677 ,  422.27844 ]],

        [[ 289.00656 ,  508.748   ]],

        [[ 274.17032 ,  603.57996 ]]],


       [[[ 283.122   ,   91.89903 ]],

        [[ 355.70074 ,  110.426704]],

        [[ 426.5647  ,  128.01099 ]],

        ...,

        [[ 469.68762 ,  582.23267 ]],

        [[ 543.2684  ,  598.20575 ]],

        [[ 616.89154 ,  614.28094 ]]],


       [[[ 617.27765 ,  538.2804  ]],

        [[ 625.3559  ,  468.35788 ]],

        [[ 633.24097 ,  398.21204 ]],

        ...,

        [[1114.9911  ,  310.21176 ]],

        [[1129.7618  ,  232.4571  ]],

        [[1145.5946  ,  151.83626 ]]],


       ...,


       [[[ 633.70874 ,  112.355156]],

        [[ 711.6895  ,  118.07471 ]],

        [[ 790.6989  ,  123.79872 ]],

        ...,

        [[ 898.90265 ,  581.32043 ]],

        [[ 974.10046 ,  588.6061  ]],

        [[1049.3226  ,  596

In [12]:
# TODO
imgpoints = []
for corners in valid_corners:
    imgpoints.append(corners)
    
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points,imgpoints,(320,240),None,None)

extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [13]:
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[1.42806727e+03 0.00000000e+00 6.28227685e+02]
 [0.00000000e+00 1.43121937e+03 3.72073582e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[ 5.97919025e-02 -3.92112082e-01  1.24165183e-03 -3.75933515e-03
   1.56092353e+00]]
Root mean squared reprojection error:
 0.5733208305621582
